In [108]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd 
import import_ipynb
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from  my_functions import detect_outliers;
import numpy as np
path = str(os.getcwd()) + '\\Data\\'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
IDtest = test["PassengerId"]
# detect outliers from Age, SibSp , Parch and Fare
Outliers_to_drop = detect_outliers(train,2,["Age","SibSp","Parch","Fare"])# check  - there is train. May be better dataset? 
train.drop(Outliers_to_drop, axis = 0).reset_index(drop=True);

In [109]:
train_len = len(train) # объединение тестовой и тренировочной выборок
print(train_len)
dataset =  pd.concat(objs=[train, test], axis=0).reset_index(drop=True)
dataset = dataset.fillna(np.nan);
dataset["Fare"] = dataset["Fare"].map(lambda i: np.log(i) if i > 0 else 0)
dataset["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in dataset['Cabin'] ]) # добавление X как идентификатора для пассажиров,
#у которых отсутствут каюта
Ticket = [] # поиск префиксса у билета, если таковой отстствует, тогда создание своего префикса
for i in list(dataset.Ticket):
    if not i.isdigit() :
        Ticket.append(i.replace(".","").replace("/","").strip().split(' ')[0]) #Take prefix
    else:
        Ticket.append("X")
dataset["Ticket"] = Ticket
dataset["Pclass"] = dataset["Pclass"].astype("category")


891


In [110]:

dataset["Embarked"] = dataset["Embarked"].fillna("S")
dataset["Sex"] = dataset["Sex"].map({"male": 0, "female":1})
dataset_title = [i.split(",")[1].split(".")[0].strip() for i in dataset["Name"]]
dataset["Title"] = pd.Series(dataset_title)
dataset["Title"] = dataset["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
dataset["Title"] = dataset["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
dataset["Title"] = dataset["Title"].astype(int)
dataset.drop(labels = ["Name"], axis = 1, inplace = True)

In [111]:
# определение размера семьи, разделение на три категории
dataset["Fsize"] = dataset["SibSp"] + dataset["Parch"] + 1
dataset['Single'] = dataset['Fsize'].map(lambda s: 1 if s == 1 else 0)
dataset['SmallF'] = dataset['Fsize'].map(lambda s: 1 if  s == 2  else 0)
dataset['MedF'] = dataset['Fsize'].map(lambda s: 1 if 3 <= s <= 4 else 0)
dataset['LargeF'] = dataset['Fsize'].map(lambda s: 1 if s >= 5 else 0)
dataset.head()

,Age,Cabin,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,Fsize,Single,SmallF,MedF,LargeF
0,22.0,X,S,1.981001,0,1,3,0,1,0.0,A5,2,2,0,1,0,0
1,38.0,C,C,4.266662,0,2,1,1,1,1.0,PC,1,2,0,1,0,0
2,26.0,X,S,2.070022,0,3,3,1,0,1.0,STONO2,1,1,1,0,0,0
3,35.0,C,S,3.972177,0,4,1,1,1,1.0,X,1,2,0,1,0,0
4,35.0,X,S,2.085672,0,5,3,0,0,0.0,X,2,1,1,0,0,0


In [112]:
# работа с заполнением пустых значений для age 
index_NaN_age = list(dataset["Age"][dataset["Age"].isnull()].index)
for i in index_NaN_age :
    age_med = dataset["Age"].median()
    age_pred = dataset["Age"][((dataset['SibSp'] == dataset.iloc[i]["SibSp"]) & (dataset['Parch'] == dataset.iloc[i]["Parch"]) & (dataset['Pclass'] == dataset.iloc[i]["Pclass"]))].median()
    if not np.isnan(age_pred) :
        dataset['Age'].iloc[i] = age_pred
    else :
        dataset['Age'].iloc[i] = age_med
#Continuous variable bins; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
#Fare Bins/Buckets using qcut or frequency bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html

In [113]:
# пробую использовать дамми кодирование, лучше поробовать использовать свое 
dataset = pd.get_dummies(dataset, columns = ["Title"])
dataset = pd.get_dummies(dataset, columns = ["Embarked"], prefix="Em")
dataset = pd.get_dummies(dataset, columns = ["Cabin"],prefix="Cabin")
dataset = pd.get_dummies(dataset, columns = ["Ticket"], prefix="T")
dataset = pd.get_dummies(dataset, columns = ["Pclass"],prefix="Pc")


In [114]:
#dataset.drop(labels = ["PassengerId"], axis = 1, inplace = True)
dataset.head()

,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,Fsize,Single,SmallF,...,T_STONO,T_STONO2,T_STONOQ,T_SWPP,T_WC,T_WEP,T_X,Pc_1,Pc_2,Pc_3
0,22.0,1.981001,0,1,0,1,0.0,2,0,1,...,0,0,0,0,0,0,0,0,0,1
1,38.0,4.266662,0,2,1,1,1.0,2,0,1,...,0,0,0,0,0,0,0,1,0,0
2,26.0,2.070022,0,3,1,0,1.0,1,1,0,...,0,1,0,0,0,0,0,0,0,1
3,35.0,3.972177,0,4,1,1,1.0,2,0,1,...,0,0,0,0,0,0,1,1,0,0
4,35.0,2.085672,0,5,0,0,0.0,1,1,0,...,0,0,0,0,0,0,1,0,0,1


In [115]:
dataset.to_csv("clean_ds.csv",index=False)
dataset

,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,Fsize,Single,SmallF,...,T_STONO,T_STONO2,T_STONOQ,T_SWPP,T_WC,T_WEP,T_X,Pc_1,Pc_2,Pc_3
0,22.0,1.981001,0,1,0,1,0.0,2,0,1,...,0,0,0,0,0,0,0,0,0,1
1,38.0,4.266662,0,2,1,1,1.0,2,0,1,...,0,0,0,0,0,0,0,1,0,0
2,26.0,2.070022,0,3,1,0,1.0,1,1,0,...,0,1,0,0,0,0,0,0,0,1
3,35.0,3.972177,0,4,1,1,1.0,2,0,1,...,0,0,0,0,0,0,1,1,0,0
4,35.0,2.085672,0,5,0,0,0.0,1,1,0,...,0,0,0,0,0,0,1,0,0,1
5,25.0,2.135148,0,6,0,0,0.0,1,1,0,...,0,0,0,0,0,0,1,0,0,1
6,54.0,3.948596,0,7,0,0,0.0,1,1,0,...,0,0,0,0,0,0,1,1,0,0
7,2.0,3.048088,1,8,0,3,0.0,5,0,0,...,0,0,0,0,0,0,1,0,0,1
8,27.0,2.409941,2,9,1,0,1.0,3,0,0,...,0,0,0,0,0,0,1,0,0,1
9,14.0,3.403555,0,10,1,1,1.0,2,0,1,...,0,0,0,0,0,0,1,0,1,0
